In [21]:
import numpy as np
from random import random

from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import QControlBox
from pytket.circuit import CircBox
from pytket.passes import DecomposeBoxes

In [ ]:
## DO NOT RUN, vanilla phase estimation from hadnout code
raise NameError
def build_phase_estimation_circuit(
    n_measurement_qubits: int, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", n_measurement_qubits)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)

    # Add Hadamard gates to every qubit in the measurement register
    for m_qubit in measurement_register:
        qpe_circ.H(m_qubit)

    # Add all (2**n_measurement_qubits - 1) of the controlled unitaries sequentially
    for m_qubit in range(n_measurement_qubits):
        control_index = n_measurement_qubits - m_qubit - 1
        control_qubit = [measurement_register[control_index]]
        for _ in range(2**m_qubit):
            qpe_circ.add_qcontrolbox(
                controlled_u_gate, control_qubit + list(state_prep_register)
            )

    # Finally, append the inverse qft and measure the qubits
    qft_box = CircBox(build_qft_circuit(n_measurement_qubits))
    inverse_qft_box = qft_box.dagger
    qpe_circ.add_circbox(inverse_qft_box, list(measurement_register))
    qpe_circ.measure_register(measurement_register, "c")

    DecomposeBoxes().apply(qpe_circ)
    
    return qpe_circ

In [48]:
def build_iter_estimation_circuit(
    m, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # create base of circuit to return
    iter_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    # the stuff you WILL eventually measure
    measurement_register = iter_circ.add_q_register("m", 1)
    state_prep_register = iter_circ.add_q_register("p", n_state_prep_qubits)

    # NEW in iterative
    classical_register = iter_circ.add_c_register("c", m)
    iter_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
    
    # Add Hadamard gates to every qubit in the measurement register
    for m_qubit in measurement_register:
        iter_circ.H(m_qubit)

    DecomposeBoxes().apply(iter_circ)

    return iter_circ

## Call the function, run experiments and stuff

In [49]:
prep_circuit = Circuit(1).X(0)  # prepare the |1> eigenstate of U1
input_angle = 0.73  # angle as number of half turns
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops
qpe_circ = build_iter_estimation_circuit(4,
    state_prep_circuit=prep_circuit, unitary_circuit=unitary_circuit
)

In [50]:
render_circuit_jupyter(qpe_circ)